# Python 类特殊方法 (魔术方法/Dunder 方法) 教程

欢迎来到 Python 类特殊方法（也称为魔术方法或 Dunder 方法）的教程！这些方法以双下划线开头和结尾（例如 `__init__`, `__str__`），它们是 Python 数据模型的关键组成部分。通过实现这些方法，你可以让你的自定义对象表现得像内置类型一样，支持迭代、算术运算、属性访问控制等。

**为什么叫 Dunder 方法？**
"Dunder" 是 "Double Underscore" 的缩写。

**为什么它们很重要？**

1.  **使自定义对象更 Pythonic**：让你的类与 Python 的内置函数和操作符（如 `len()`, `+`, `[]`）无缝集成。
2.  **协议实现**：它们定义了 Python 中的各种协议，例如迭代器协议、序列协议、数字协议等。
3.  **代码可读性**：使用标准操作符（如 `a + b` 而不是 `a.add(b)`）通常更易读。
4.  **框架和库的构建**：许多库（如 ORM、数据分析库）依赖于这些方法来提供丰富的功能。

本教程将按功能分类介绍一些最常用和最重要的魔术方法。

## 1. 基本的魔术方法

### 1.1 `__init__(self, ...)`: 对象初始化
*   **目的**：当类的实例被创建后，用于初始化该实例的属性。它不是构造函数（`__new__` 才是），而是初始化器。
*   **调用时机**：在 `__new__` 创建实例后自动调用。
*   `self` 是新创建的实例。

In [ ]:
class Book:
    def __init__(self, title, author, pages):
        print(f"Book.__init__ called for '{title}'")
        self.title = title
        self.author = author
        self.pages = pages

book1 = Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams", 224)
print(f"Title: {book1.title}, Author: {book1.author}")

### 1.2 `__new__(cls, ...)`: 对象实例化 (构造)
*   **目的**：真正的构造函数，负责创建类的实例。它在 `__init__` 之前被调用。
*   **调用时机**：当调用类名创建实例时（如 `MyClass()`）。
*   **返回值**：必须返回新创建的实例对象。如果 `__new__` 返回的不是 `cls` 的实例，那么该实例的 `__init__` 方法就不会被调用。
*   **用途**：通常用于控制实例的创建过程，例如实现单例模式，或者创建不可变类型的子类。
*   `cls` 是正在被实例化的类。

In [ ]:
class PositiveInteger(int): # 继承自不可变类型 int
    def __new__(cls, value):
        print(f"PositiveInteger.__new__ called with value: {value}")
        if not isinstance(value, int) or value < 0:
            raise ValueError("PositiveInteger must be a non-negative integer")
        # 调用父类的 __new__ 来创建实例
        instance = super().__new__(cls, value)
        print(f"  Instance created: {instance}, type: {type(instance)}")
        return instance

    # __init__ 通常不需要为不可变类型定义，因为它们在 __new__ 中已经完全形成
    # def __init__(self, value):
    #     print(f"PositiveInteger.__init__ called with self: {self}, value: {value}")
    #     # super().__init__() # int 的 __init__ 不需要参数

p_int1 = PositiveInteger(10)
print(f"Value of p_int1: {p_int1}, p_int1 + 5 = {p_int1 + 5}")

try:
    p_int2 = PositiveInteger(-5)
except ValueError as e:
    print(f"Error: {e}")

# 单例模式示例 (简单版)
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            print("Singleton: Creating new instance")
            cls._instance = super().__new__(cls)
        else:
            print("Singleton: Returning existing instance")
        return cls._instance
    
    def __init__(self, data=None):
        # __init__ 总是会被调用，即使返回的是旧实例
        # 所以，初始化逻辑需要小心处理，确保只在第一次创建时执行
        if not hasattr(self, 'initialized'):
            print(f"Singleton.__init__ called (first time) with data: {data}")
            self.data = data
            self.initialized = True
        else:
            print(f"Singleton.__init__ called (already initialized), current data: {self.data}, new data ignored: {data}")

s1 = Singleton("Data for S1")
s2 = Singleton("Data for S2") # __init__ 会被调用，但 s1 和 s2 是同一个实例
print(f"s1 is s2: {s1 is s2}")
print(f"s1.data: {s1.data}") # 'Data for S1'
print(f"s2.data: {s2.data}") # 'Data for S1'

### 1.3 `__del__(self)`: 对象销毁 (析构)
*   **目的**：当对象即将被垃圾回收器销毁时调用。用于执行清理操作，如关闭文件、释放资源等。
*   **调用时机**：不确定，由垃圾回收器决定。不应该依赖它来执行关键的清理操作（使用 `try...finally` 或上下文管理器更好）。
*   **警告**：`__del__` 的行为有时难以预测，尤其是在存在循环引用的情况下。应谨慎使用。

In [ ]:
class FileHandler:
    def __init__(self, filename):
        print(f"FileHandler: Opening {filename}")
        self.filename = filename
        self.file = open(filename, 'w') # 示例，实际应更健壮
        
    def write(self, content):
        self.file.write(content)
        
    def __del__(self):
        # 尽量确保文件关闭，但这不是最佳实践
        if hasattr(self, 'file') and not self.file.closed:
            print(f"FileHandler.__del__: Closing {self.filename}")
            self.file.close()

def create_and_delete_handler():
    print("Creating FileHandler...")
    fh = FileHandler("temp_del_test.txt")
    fh.write("Hello from __del__ test\n")
    print("FileHandler created and used. Exiting function scope...")
    # 当 fh 离开作用域且没有其他引用时，它可能被垃圾回收

create_and_delete_handler()
print("After function call. __del__ might have been called (or might be later).")

# 为了确保能看到 __del__ 的输出，可以显式删除并触发垃圾回收 (不推荐在生产代码中)
# import gc
# fh_explicit = FileHandler("temp_del_explicit.txt")
# del fh_explicit
# gc.collect()

## 2. 对象表示的魔术方法

### 2.1 `__str__(self)`: 对象的可读字符串表示
*   **目的**：返回一个对象的“非正式”或“用户友好”的字符串表示。
*   **调用时机**：当使用 `str(obj)` 或 `print(obj)` 时。
*   **返回值**：必须是一个字符串对象。

### 2.2 `__repr__(self)`: 对象的“官方”字符串表示
*   **目的**：返回一个对象的“官方”或“开发者友好”的字符串表示，理想情况下，这个字符串应该是一个有效的 Python 表达式，可以用来重新创建具有相同值的对象（`eval(repr(obj)) == obj`）。如果不可能，至少应该提供足够的信息来识别对象。
*   **调用时机**：当使用 `repr(obj)`，在交互式解释器中直接输入变量名，或者当 `__str__` 未定义时 `str()` 和 `print()` 也会回退到它。
*   **返回值**：必须是一个字符串对象。
*   **最佳实践**：如果实现了 `__repr__` 但未实现 `__str__`，`str()` 会使用 `__repr__`。通常建议至少实现 `__repr__`。

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        # 用户友好的表示
        return f"Point at ({self.x}, {self.y})"

    def __repr__(self):
        # 开发者友好的表示，理想情况下可用于重建对象
        return f"Point(x={self.x!r}, y={self.y!r})" # 使用 !r 确保内部值也是 repr 形式

p = Point(3, 4)
print(str(p))    # 调用 p.__str__()
print(repr(p))   # 调用 p.__repr__()
print(p)         # 在 print 中，优先调用 __str__

# 在解释器中直接输入变量名，会调用 __repr__
# (在Jupyter Notebook单元格的最后一行输出，也是 __repr__)
p

### 2.3 `__format__(self, format_spec)`: 自定义格式化
*   **目的**：允许对象自定义其在格式化字符串（如 f-string 或 `str.format()`）中的表示。
*   **调用时机**：当使用 `format(obj, format_spec)` 或在格式化字符串中使用格式说明符时（例如 `f"{obj:spec}"`）。
*   `format_spec` 是格式说明符字符串。

In [ ]:
from datetime import date

class MyDate(date):
    def __format__(self, format_spec):
        if not format_spec:
            format_spec = "%Y-%m-%d" # 默认格式
        elif format_spec == 'mdy':
            format_spec = "%m/%d/%y"
        elif format_spec == 'dmy':
            format_spec = "%d-%m-%Y"
        # 如果不是我们支持的特殊格式，就调用父类的 __format__
        # 或者可以直接使用 self.strftime(format_spec)
        return self.strftime(format_spec)

today = MyDate(2023, 10, 26)
print(f"Default format: {today}") # 这里的 today 会先尝试 __str__
print(f"Default format (explicit): {today:}")
print(f"MDY format: {today:mdy}")
print(f"DMY format: {today:dmy}")
print(f"ISO format: {today:%Y%m%d}") # 使用标准的 strftime 格式

## 3. 属性访问的魔术方法

### 3.1 `__getattr__(self, name)`: 访问不存在的属性
*   **目的**：当尝试访问一个实例中不存在的属性时被调用。
*   **调用时机**：仅当正常的属性查找失败后（即属性不在实例的 `__dict__` 中，也不在类及其基类的 `__dict__` 中，也不是描述符）。
*   **用途**：用于实现属性的动态计算、代理、延迟加载等。
*   如果找不到属性，应引发 `AttributeError`。

### 3.2 `__getattribute__(self, name)`: 无条件属性访问
*   **目的**：无条件地拦截所有属性访问（即使属性存在）。
*   **调用时机**：每次访问实例属性时都会调用。
*   **警告**：实现 `__getattribute__` 时要非常小心，因为它很容易导致无限递归（例如，在 `__getattribute__` 内部直接访问 `self.attr` 会再次调用 `__getattribute__`）。通常需要通过 `super().__getattribute__(name)` 或直接访问 `object.__getattribute__(self, name)` 来获取属性值。
*   如果同时定义了 `__getattr__` 和 `__getattribute__`，`__getattr__` 只有在 `__getattribute__` 引发 `AttributeError` 时才会被调用（或者 `__getattribute__` 显式调用它）。

### 3.3 `__setattr__(self, name, value)`: 设置属性
*   **目的**：当尝试给实例属性赋值时调用。
*   **调用时机**：每次执行 `obj.name = value` 时。
*   **警告**：类似于 `__getattribute__`，在 `__setattr__` 内部直接使用 `self.name = value` 会导致无限递归。通常使用 `super().__setattr__(name, value)` 或 `object.__setattr__(self, name, value)` 或直接操作 `self.__dict__`。

### 3.4 `__delattr__(self, name)`: 删除属性
*   **目的**：当尝试删除实例属性时调用。
*   **调用时机**：每次执行 `del obj.name` 时。
*   **警告**：类似地，直接在内部使用 `del self.name` 会导致无限递归。通常使用 `super().__delattr__(name)` 或 `object.__delattr__(self, name)` 或直接操作 `self.__dict__`。

### 3.5 `__dir__(self)`: 列出属性
*   **目的**：当调用 `dir(obj)` 时，返回一个包含对象属性名称的列表（字符串）。
*   **用途**：自定义 `dir()` 的输出，例如包含动态生成的属性。

In [ ]:
class DynamicAttributes:
    def __init__(self, initial_data=None):
        print("DynamicAttributes.__init__")
        # 使用 object.__setattr__ 避免触发我们自己的 __setattr__ (如果已定义)
        object.__setattr__(self, '_data', initial_data or {})
        object.__setattr__(self, '_dynamic_prefix', "dyn_")

    def __getattr__(self, name):
        print(f"DynamicAttributes.__getattr__ called for '{name}'")
        if name.startswith(self._dynamic_prefix):
            actual_key = name[len(self._dynamic_prefix):]
            if actual_key in self._data:
                return self._data[actual_key]
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")

    def __setattr__(self, name, value):
        print(f"DynamicAttributes.__setattr__ called for '{name}' = {value!r}")
        if name.startswith(self._dynamic_prefix):
            actual_key = name[len(self._dynamic_prefix):]
            self._data[actual_key] = value
        else:
            # 对于非动态属性，委托给父类 (object) 的 __setattr__
            super().__setattr__(name, value)

    def __delattr__(self, name):
        print(f"DynamicAttributes.__delattr__ called for '{name}'")
        if name.startswith(self._dynamic_prefix):
            actual_key = name[len(self._dynamic_prefix):]
            if actual_key in self._data:
                del self._data[actual_key]
                return
        super().__delattr__(name) # 如果不是动态属性或不存在，让父类处理或报错

    def __dir__(self):
        print("DynamicAttributes.__dir__ called")
        # 合并常规属性和动态属性
        default_dir = super().__dir__()
        dynamic_attrs = [self._dynamic_prefix + key for key in self._data.keys()]
        return sorted(list(set(default_dir + dynamic_attrs)))

    # __getattribute__ 示例 (谨慎使用)
    # def __getattribute__(self, name):
    #     print(f"DynamicAttributes.__getattribute__ for '{name}'")
    #     # 必须非常小心以避免无限递归
    #     # 例如，总是通过 super() 访问属性
    #     try:
    #         return super().__getattribute__(name)
    #     except AttributeError:
    #         # 如果常规查找失败，可以尝试我们的动态逻辑
    #         if name.startswith(object.__getattribute__(self, '_dynamic_prefix')):
    #             _data = object.__getattribute__(self, '_data') # 安全获取 _data
    #             _dynamic_prefix = object.__getattribute__(self, '_dynamic_prefix')
    #             actual_key = name[len(_dynamic_prefix):]
    #             if actual_key in _data:
    #                 return _data[actual_key]
    #         raise # 重新抛出原始 AttributeError

dyn = DynamicAttributes({"version": "1.0", "status": "active"})
dyn.normal_attr = 100 # 调用 __setattr__

print(f"\nAccessing normal_attr: {dyn.normal_attr}") # 正常访问，不走 __getattr__
print(f"Accessing dyn_version: {dyn.dyn_version}")     # 调用 __getattr__
print(f"Accessing dyn_status: {dyn.dyn_status}")

dyn.dyn_user = "admin" # 调用 __setattr__
print(f"Accessing dyn_user: {dyn.dyn_user}")

try:
    print(dyn.non_existent)
except AttributeError as e:
    print(f"Error accessing non_existent: {e}")

print("\n--- Before del --- ")
print(dir(dyn)) # 调用 __dir__
del dyn.dyn_status # 调用 __delattr__
print("\n--- After del dyn_status --- ")
print(dir(dyn))

try:
    print(dyn.dyn_status)
except AttributeError as e:
    print(f"Error after deleting dyn_status: {e}")

## 4. 比较的魔术方法

这些方法用于实现对象间的比较操作符。
*   `__eq__(self, other)`: `self == other`
*   `__ne__(self, other)`: `self != other` (如果未实现，默认是 `not (self == other)`)
*   `__lt__(self, other)`: `self < other`
*   `__le__(self, other)`: `self <= other`
*   `__gt__(self, other)`: `self > other`
*   `__ge__(self, other)`: `self >= other`

**注意**：
*   如果只实现 `__eq__` 而不实现 `__ne__`，Python 会自动提供 `__ne__` 的默认实现。
*   对于富比较方法 (`__lt__`, `__le__`, `__gt__`, `__ge__`)，如果你实现了一个，通常也应该实现它的反面（例如，实现了 `__lt__`，也应该考虑 `__gt__`）。
*   `functools.total_ordering` 装饰器：如果你实现了 `__eq__` 和至少一个富比较方法（如 `__lt__`），这个装饰器可以自动为你生成其他所有富比较方法。

In [ ]:
import functools

@functools.total_ordering # 自动生成其他比较方法
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __eq__(self, other):
        print(f"Person.__eq__ called for {self.name} == {other.name if isinstance(other, Person) else other}")
        if not isinstance(other, Person):
            return NotImplemented # 重要：让 Python 尝试 other.__eq__(self)
        return self.name == other.name and self.age == other.age

    def __lt__(self, other):
        print(f"Person.__lt__ called for {self.name} < {other.name if isinstance(other, Person) else other}")
        if not isinstance(other, Person):
            return NotImplemented
        # 首先按年龄比较，如果年龄相同，按名字字母顺序比较
        if self.age != other.age:
            return self.age < other.age
        return self.name < other.name
    
    def __repr__(self):
        return f"Person('{self.name}', {self.age})"

p1 = Person("Alice", 30)
p2 = Person("Bob", 25)
p3 = Person("Alice", 30)
p4 = Person("Charlie", 30)

print(f"p1 == p2: {p1 == p2}") # False
print(f"p1 == p3: {p1 == p3}") # True
print(f"p1 != p2: {p1 != p2}") # True (自动生成或默认)

print(f"p1 < p2: {p1 < p2}")   # False (30 < 25 is False)
print(f"p2 < p1: {p2 < p1}")   # True  (25 < 30 is True)
print(f"p1 > p2: {p1 > p2}")   # True (由 total_ordering 生成)
print(f"p1 <= p3: {p1 <= p3}") # True (由 total_ordering 生成)
print(f"p1 < p4: {p1 < p4}")   # True (Alice < Charlie, age is same)

persons = [p1, p2, p4, Person("David", 25)]
print(f"Original list: {persons}")
persons.sort() # sort() 使用了 __lt__
print(f"Sorted list: {persons}")

### 4.1 `__hash__(self)`: 哈希值
*   **目的**：返回对象的哈希值 (一个整数)。用于将对象放入哈希表结构中，如字典的键或集合的元素。
*   **规则**：
    1.  如果 `a == b`，那么必须 `hash(a) == hash(b)`。
    2.  如果对象是可变的（其影响哈希值的部分可以改变），那么它不应该实现 `__hash__`，或者 `__hash__` 应该被设置为 `None`（这是Python 3中可变类的默认行为）。
    3.  如果只实现了 `__eq__` 而没有实现 `__hash__`，类的实例默认是不可哈希的 (`__hash__` 会被隐式设为 `None`)。
    4.  如果想让一个实现了 `__eq__` 的类仍然使用其父类的 `__hash__` (例如 `object.__hash__`，它基于对象ID)，可以显式设置 `__hash__ = <ParentClass>.__hash__`。
*   通常，哈希值是基于对象中不可变属性的元组来计算的，例如 `hash((self.attr1, self.attr2))`。

In [ ]:
class ImmutablePoint:
    def __init__(self, x, y):
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    def __eq__(self, other):
        if not isinstance(other, ImmutablePoint):
            return NotImplemented
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        # 基于不可变属性计算哈希值
        print(f"ImmutablePoint.__hash__ called for ({self.x}, {self.y})")
        return hash((self.x, self.y))

    def __repr__(self):
        return f"ImmutablePoint({self.x}, {self.y})"

ip1 = ImmutablePoint(1, 2)
ip2 = ImmutablePoint(1, 2)
ip3 = ImmutablePoint(3, 4)

print(f"ip1 == ip2: {ip1 == ip2}") # True
print(f"hash(ip1): {hash(ip1)}")
print(f"hash(ip2): {hash(ip2)}") # 应该与 hash(ip1) 相同
print(f"hash(ip3): {hash(ip3)}")

point_set = {ip1, ip2, ip3}
print(f"Set of points: {point_set}") # {ImmutablePoint(1, 2), ImmutablePoint(3, 4)}

point_dict = {ip1: "Origin related", ip3: "Other point"}
print(f"Dict with points as keys: {point_dict}")

## 5. 模拟数值类型的魔术方法

这些方法允许你的对象参与算术运算。

### 5.1 一元运算符和函数:
*   `__pos__(self)`: `+self`
*   `__neg__(self)`: `-self`
*   `__abs__(self)`: `abs(self)`
*   `__invert__(self)`: `~self` (按位取反)
*   `__round__(self, ndigits)`: `round(self, ndigits)`
*   `__floor__(self)`: `math.floor(self)`
*   `__ceil__(self)`: `math.ceil(self)`
*   `__trunc__(self)`: `math.trunc(self)`

### 5.2 类型转换:
*   `__bool__(self)`: `bool(self)` (对象在布尔上下文中的真值，如 `if obj:`)。如果未定义，则会检查 `__len__`。
*   `__int__(self)`: `int(self)`
*   `__float__(self)`: `float(self)`
*   `__complex__(self)`: `complex(self)`
*   `__index__(self)`: 当对象被用作序列索引时（如 `my_list[obj]`），Python 内部会尝试调用它来获取一个整数索引。如果未实现，则会尝试 `__int__`。主要用于确保对象可以安全地用作索引。

### 5.3 二元算术运算符:
对于每个二元运算符 `op` (如 `+`, `-`, `*`, `/`, `//`, `%`, `**`, `<<`, `>>`, `&`, `^`, `|`)，都有三个相关的魔术方法：
1.  **标准 (Normal)**: `__op__(self, other)` (例如 `__add__(self, other)` for `self + other`)
2.  **反射 (Reflected)**: `__rop__(self, other)` (例如 `__radd__(self, other)` for `other + self`)。
    *   当 `other + self` 时，如果 `other` 没有实现 `__add__` 或者其 `__add__` 对 `self` 的类型返回 `NotImplemented`，Python 会尝试调用 `self.__radd__(other)`。
3.  **原地 (In-place)**: `__iop__(self, other)` (例如 `__iadd__(self, other)` for `self += other`)
    *   如果实现了，它会直接修改 `self` 并返回 `self`。
    *   如果未实现 `__iadd__`，`self += other` 会回退到 `self = self.__add__(other)`。

**例子：**
*   `__add__(self, other)`: `self + other`
*   `__sub__(self, other)`: `self - other`
*   `__mul__(self, other)`: `self * other`
*   `__matmul__(self, other)`: `self @ other` (Python 3.5+ 矩阵乘法)
*   `__truediv__(self, other)`: `self / other` (真除法)
*   `__floordiv__(self, other)`: `self // other` (地板除)
*   `__mod__(self, other)`: `self % other`
*   `__pow__(self, other[, modulo])`: `self ** other` or `pow(self, other, modulo)`
*   `__lshift__(self, other)`: `self << other`
*   `__rshift__(self, other)`: `self >> other`
*   `__and__(self, other)`: `self & other`
*   `__xor__(self, other)`: `self ^ other`
*   `__or__(self, other)`: `self | other`

对应的反射版本是 `__radd__`, `__rsub__`, etc. 和原地版本 `__iadd__`, `__isub__`, etc.

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

    def __str__(self):
        return f"({self.x}, {self.y})"

    def __add__(self, other):
        print("Vector.__add__ called")
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        elif isinstance(other, (int, float)):
            return Vector(self.x + other, self.y + other)
        return NotImplemented

    def __radd__(self, other):
        # other + self, 当 other 不是 Vector 或其 __add__ 返回 NotImplemented 时调用
        print("Vector.__radd__ called")
        # 这里的逻辑通常和 __add__ 中处理标量的情况类似
        return self.__add__(other) # 可以直接复用 __add__

    def __iadd__(self, other):
        print("Vector.__iadd__ called")
        if isinstance(other, Vector):
            self.x += other.x
            self.y += other.y
            return self # 原地操作必须返回 self
        elif isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self
        return NotImplemented

    def __mul__(self, scalar):
        print("Vector.__mul__ called")
        if isinstance(scalar, (int, float)):
            return Vector(self.x * scalar, self.y * scalar)
        return NotImplemented

    def __rmul__(self, scalar):
        print("Vector.__rmul__ called")
        return self.__mul__(scalar) # 乘法通常是可交换的

    def __len__(self):
        # 随便定义一个长度，比如维度
        print("Vector.__len__ called")
        return 2

    def __bool__(self):
        # 如果向量不是零向量，则为 True
        print("Vector.__bool__ called")
        return self.x != 0 or self.y != 0

v1 = Vector(1, 2)
v2 = Vector(3, 4)

print(f"v1 + v2 = {v1 + v2}") # (4, 6)
print(f"v1 + 10 = {v1 + 10}") # (11, 12)
print(f"10 + v1 = {10 + v1}") # (11, 12) - 调用 __radd__

v_orig_id = id(v1)
v1 += v2
print(f"v1 after += v2: {v1}") # (4, 6)
print(f"id(v1) changed: {id(v1) != v_orig_id}") # False, 因为 __iadd__ 返回 self

v3 = Vector(2,3)
print(f"v3 * 3 = {v3 * 3}")     # (6,9)
print(f"3 * v3 = {3 * v3}")     # (6,9) - 调用 __rmul__

print(f"len(v1): {len(v1)}")
print(f"bool(Vector(0,0)): {bool(Vector(0,0))}")
print(f"bool(Vector(1,0)): {bool(Vector(1,0))}")

## 6. 模拟容器类型的魔术方法 (集合协议)

这些方法使你的对象表现得像列表、字典或集合等容器。

*   `__len__(self)`: `len(self)`。返回容器中元素的数量。如果一个类没有定义 `__bool__`，但定义了 `__len__`，那么当 `len(obj) > 0` 时，`bool(obj)` 为 `True`。
*   `__getitem__(self, key)`: `self[key]`。获取键对应的项。
*   `__setitem__(self, key, value)`: `self[key] = value`。设置键对应的项。
*   `__delitem__(self, key)`: `del self[key]`。删除键对应的项。
*   `__iter__(self)`: `iter(self)`。应返回一个迭代器对象。
*   `__reversed__(self)`: `reversed(self)`。应返回一个反向迭代器对象 (可选)。
*   `__contains__(self, item)`: `item in self`。检查 `item` 是否在容器中。如果未实现，Python 会迭代容器并逐个比较。

如果实现了 `__iter__`，你的对象就是**可迭代的 (iterable)**。
如果实现了 `__getitem__` 和 `__len__`，你的对象就是**序列 (sequence)**，并且也会自动支持迭代和 `in` 操作符（尽管显式实现 `__iter__` 和 `__contains__` 可能更高效）。

In [ ]:
class MyCustomList:
    def __init__(self, initial_data=None):
        self._data = list(initial_data) if initial_data else []

    def __len__(self):
        print("MyCustomList.__len__ called")
        return len(self._data)

    def __getitem__(self, index):
        print(f"MyCustomList.__getitem__ called with index: {index}")
        if isinstance(index, slice):
            # 处理切片
            return MyCustomList(self._data[index])
        return self._data[index]

    def __setitem__(self, index, value):
        print(f"MyCustomList.__setitem__ called with index: {index}, value: {value}")
        self._data[index] = value

    def __delitem__(self, index):
        print(f"MyCustomList.__delitem__ called with index: {index}")
        del self._data[index]

    def __iter__(self):
        print("MyCustomList.__iter__ called")
        # yield from self._data # 简单的迭代器
        return iter(self._data) # 或者返回内置列表的迭代器

    def __contains__(self, item):
        print(f"MyCustomList.__contains__ called for item: {item}")
        return item in self._data

    def __repr__(self):
        return f"MyCustomList({self._data!r})"

    def append(self, item):
        self._data.append(item)

my_list = MyCustomList([1, 2, 3])
print(f"Length: {len(my_list)}")
print(f"Element at index 1: {my_list[1]}")
my_list[0] = 10
print(f"After setting index 0: {my_list}")
del my_list[2]
print(f"After deleting index 2: {my_list}")

print("\nIterating over the list:")
for item in my_list:
    print(item)

print(f"\nIs 10 in my_list? {10 in my_list}")
print(f"Is 5 in my_list? {5 in my_list}")

sliced = my_list[0:1]
print(f"Slice [0:1]: {sliced}, type: {type(sliced)}")
my_list.append(20)
print(f"After append: {my_list}")

## 7. 可调用对象 (Callable Objects)

### 7.1 `__call__(self, *args, **kwargs)`
*   **目的**：允许类的实例像函数一样被调用。
*   **调用时机**：当对实例使用函数调用语法时，如 `obj(*args, **kwargs)`。

In [ ]:
class Adder:
    def __init__(self, base_value):
        self.base_value = base_value
        print(f"Adder initialized with base_value: {base_value}")

    def __call__(self, x, y):
        print(f"Adder.__call__ with x={x}, y={y}, base_value={self.base_value}")
        return self.base_value + x + y

add_5 = Adder(5)
result = add_5(10, 20) # 调用 add_5.__call__(10, 20)
print(f"Result: {result}")

add_100 = Adder(100)
result2 = add_100(1, 2)
print(f"Result2: {result2}")

## 8. 上下文管理器 (Context Management Protocol)

这两个方法使得对象可以与 `with` 语句一起使用，用于管理资源（如文件、锁、数据库连接）。

*   `__enter__(self)`: 进入 `with` 语句块之前调用。它的返回值（如果有的话）会赋给 `as` 子句中的变量。
*   `__exit__(self, exc_type, exc_val, exc_tb)`: 退出 `with` 语句块时调用，无论块内是否发生异常。
    *   `exc_type`, `exc_val`, `exc_tb`：如果块内发生异常，它们是异常的类型、值和回溯信息；否则都是 `None`。
    *   如果 `__exit__` 返回 `True`，则异常被“抑制”（即不会向外传播）。如果返回 `False`（或 `None`，这是默认行为），异常会继续传播。

In [ ]:
class ManagedResource:
    def __init__(self, name):
        self.name = name
        print(f"ManagedResource '{self.name}': Initialized")

    def __enter__(self):
        print(f"ManagedResource '{self.name}': __enter__ (Acquiring resource)")
        # 模拟获取资源
        return self # 返回的值赋给 'as' 后面的变量

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"ManagedResource '{self.name}': __exit__ (Releasing resource)")
        if exc_type:
            print(f"  Exception occurred: Type={exc_type}, Value={exc_val}")
            # return True # 如果返回 True，异常会被抑制
        print(f"  Resource '{self.name}' released.")
        return False # 默认行为，如果发生异常，则继续传播

    def use(self):
        print(f"  Using resource '{self.name}'...")

print("--- Context Manager: Normal execution ---")
with ManagedResource("DB Connection") as res:
    res.use()
    print("  Inside with block.")
print("After with block.")

print("\n--- Context Manager: With exception ---")
try:
    with ManagedResource("File Stream") as fs:
        fs.use()
        raise ValueError("Something went wrong inside 'with'!")
        print("  This line won't be reached.")
except ValueError as e:
    print(f"Caught expected error: {e}")
print("After with block (with exception).")

## 9. 描述符协议 (Descriptor Protocol)

描述符是实现了特定协议的类，当其作为另一个类（宿主类）的类属性时，可以自定义对该属性的访问行为。这部分内容较为高级，在元编程教程中通常会详细介绍。

*   `__get__(self, instance, owner)`: 当获取属性值时调用。
*   `__set__(self, instance, value)`: 当设置属性值时调用。
*   `__delete__(self, instance)`: 当删除属性时调用。

内置的 `property`, `@staticmethod`, `@classmethod` 都是基于描述符协议实现的。

In [ ]:
# 简单示例，详细内容请参考元编程教程
class RevealAccess:
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, instance, owner):
        print(f"Descriptor '{self.name}': __get__ from instance {instance} (owner {owner})")
        return self.val

    def __set__(self, instance, value):
        print(f"Descriptor '{self.name}': __set__ on instance {instance} to {value}")
        self.val = value

class MyClassWithDescriptor:
    x = RevealAccess(10, 'x') # x 是一个描述符实例

m = MyClassWithDescriptor()
print(f"Value of m.x: {m.x}")
m.x = 20
print(f"New value of m.x: {m.x}")

## 10. 异步编程的魔术方法 (Python 3.5+)

这些方法用于支持 `async/await` 语法。

*   `__await__(self)`: 必须返回一个迭代器。用于使对象成为**可等待对象 (awaitable)**。`async def` 函数返回的协程对象就实现了 `__await__`。
*   `__aiter__(self)`: 必须返回一个**异步迭代器 (asynchronous iterator)** 对象。用于 `async for`。
*   `__anext__(self)`: 必须返回一个可等待对象，该可等待对象在完成时产生下一个值。当没有更多项时，必须引发 `StopAsyncIteration` 异常。由异步迭代器实现。
*   `__aenter__(self)`: 类似于 `__enter__`，但用于异步上下文管理器 (`async with`)。必须返回一个可等待对象。
*   `__aexit__(self, exc_type, exc_val, exc_tb)`: 类似于 `__exit__`，但用于异步上下文管理器。必须返回一个可等待对象。

In [ ]:
import asyncio

# 异步迭代器示例
class AsyncCounter:
    def __init__(self, limit):
        self.limit = limit
        self.current = 0

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self.current < self.limit:
            await asyncio.sleep(0.1) # 模拟异步操作
            value = self.current
            self.current += 1
            return value
        else:
            raise StopAsyncIteration

# 异步上下文管理器示例
class AsyncManaged:
    async def __aenter__(self):
        print("AsyncManaged: Entering context...")
        await asyncio.sleep(0.1)
        print("AsyncManaged: Resource acquired.")
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print("AsyncManaged: Exiting context...")
        await asyncio.sleep(0.1)
        if exc_type:
            print(f"  AsyncManaged: Exception occurred: {exc_val}")
        print("AsyncManaged: Resource released.")

    async def do_work(self):
        print("  AsyncManaged: Doing work...")
        await asyncio.sleep(0.2)

async def main_async_demo():
    print("--- Async Iterator Demo ---")
    async for i in AsyncCounter(3):
        print(f"  Got: {i}")

    print("\n--- Async Context Manager Demo ---")
    async with AsyncManaged() as am:
        await am.do_work()

# 在Jupyter Notebook中运行asyncio代码
# asyncio.run(main_async_demo()) # 如果直接运行会报错，因为Jupyter已经有事件循环

# 对于Jupyter, 通常可以直接 await (如果IPython版本够新)
# 或者使用 nest_asyncio
# 为了简单起见，这里不直接运行，但代码结构是正确的
print("Async demo functions defined. Run in an async context.")

# 如果你想在 notebook cell 中直接运行, 你可以这样做:
# import nest_asyncio
# nest_asyncio.apply()
# asyncio.run(main_async_demo())

## 11. 其他一些有用的魔术属性和方法

### 魔术属性 (通常由解释器设置和使用):
*   `__doc__`: 对象的文档字符串。
*   `__name__`: 类、函数、方法的名称 (字符串)。模块的 `__name__` 在直接运行时是 `"__main__"`。
*   `__qualname__` (Python 3.3+): 类或函数的限定名称，包括其所在的命名空间路径。
*   `__module__`: 类或函数所在的模块名称 (字符串)。
*   `__class__`: 实例所属的类 (`obj.__class__` 与 `type(obj)` 相同)。
*   `__dict__`: 存储对象（或类）可写属性的字典。不是所有对象都有 `__dict__` (例如，使用 `__slots__` 的类或某些内置类型)。
*   `__slots__`: 一个字符串元组，用于声明固定的实例属性集。可以节省内存并加快属性访问速度，但实例将不再有 `__dict__` (除非 `__dict__` 明确包含在 `__slots__` 中)，也不能动态添加新属性。
*   `__bases__`: 类的父类元组。
*   `__mro__`: 方法解析顺序 (Method Resolution Order) 元组，定义了类继承和方法查找的顺序。

### 其他魔术方法:
*   `__instancecheck__(self, instance)`: `isinstance(instance, cls)`。
*   `__subclasscheck__(self, subclass)`: `issubclass(subclass, cls)`。
    *   这两个通常在元类中实现，用于自定义 `isinstance()` 和 `issubclass()` 的行为。
*   `__getstate__(self)` 和 `__setstate__(self, state)`: 用于控制对象如何被 `pickle` 模块序列化和反序列化。
*   `__copy__(self)` 和 `__deepcopy__(self, memo)`: 用于支持 `copy.copy()` 和 `copy.deepcopy()`。
*   `__class_getitem__(cls, key)` (Python 3.9+): 允许类支持泛型类型订阅，例如 `MyClass[int]`。

In [ ]:
class MySlotsClass:
    __slots__ = ('x', 'y') # 定义固定的实例属性
    # __slots__ = ('x', 'y', '__dict__') # 如果想保留 __dict__
    def __init__(self, x, y):
        self.x = x
        self.y = y

slot_obj = MySlotsClass(1, 2)
print(f"slot_obj.x: {slot_obj.x}")
# print(slot_obj.__dict__) # 会报错 AttributeError，除非 __dict__ 在 __slots__ 中
try:
    slot_obj.z = 3 # 不能动态添加属性
except AttributeError as e:
    print(f"Error adding new attribute to slots object: {e}")

print(f"\nBook class doc: {Book.__doc__ if Book.__doc__ else 'No docstring'}")
print(f"Book class name: {Book.__name__}")
print(f"Book class module: {Book.__module__}")
print(f"book1's class: {book1.__class__}")
print(f"book1's __dict__: {book1.__dict__}")
print(f"Point class bases: {Point.__bases__}")
print(f"Point class MRO: {Point.__mro__}")

# __class_getitem__ 示例 (Python 3.9+)
class GenericContainer:
    def __class_getitem__(cls, item_type):
        print(f"GenericContainer.__class_getitem__ called with item_type: {item_type}")
        # 实际上这里应该返回一个新的泛型类或别名
        # 为简单起见，我们只打印
        return f"GenericContainer specialized for {item_type}"

if hasattr(GenericContainer, '__class_getitem__'): # 检查是否在 Python 3.9+
    specialized_container = GenericContainer[int]
    print(specialized_container)
    specialized_container_str = GenericContainer[str]
    print(specialized_container_str)
else:
    print("__class_getitem__ not available in this Python version.")

## 总结

魔术方法是 Python 强大且富有表现力的特性，它们使得自定义类能够深度集成到语言的核心机制中。

**关键 takeaways：**

*   **按需实现**：你不需要实现所有的魔术方法，只需要实现那些对你的类有意义的方法。
*   **理解协议**：了解每个魔术方法属于哪个协议（如序列、数字、上下文管理）有助于理解其目的。
*   **`NotImplemented`**：在二元操作符（如 `__add__`, `__eq__`）中，如果你的方法不知道如何处理 `other` 参数的类型，应该返回 `NotImplemented`，这样 Python 可以尝试调用 `other` 对象的反射方法（如 `other.__radd__(self)`）。
*   **查阅文档**：Python 官方文档的“数据模型”章节是关于魔术方法最权威和详细的参考。

通过熟练运用这些特殊方法，你可以编写出更优雅、更 Pythonic、功能更丰富的类。